In [128]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import bisplrep, bisplev
from scipy import integrate

In [116]:
def create_knots(k,n,min=0,max=10):
    # Create open-uniform knot vector for a B-spline of degree k and with n+1 control points between the min and max values.  
    return np.concatenate(
        [(min)*np.zeros(k),
         np.linspace(0,max,n),
         (max)*np.ones(k)]
        )    

def fit_Bspline(x,y,z,k,tx,ty):
    #Fit B-spline on the given dataset of x,y,z and return its tck form and the depth-error.
    tck,depth_loss,_,_=bisplrep(x, y, z, kx=k, ky=k, task=-1, s=0, tx=tx, ty=ty, full_output=1)
    return tck, depth_loss

#def int_depth_loss(tck)

def curvature_loss(tck, min_x=0, max_x=10, min_y=0, max_y=10):
    bs_xx = lambda x,y:bisplev(x,y,tck,2,0) #Second dervivative of the B-spline w.r.t. x
    bs_yy= lambda x,y:bisplev(x,y,tck,0,2) #Second dervivative of the B-spline w.r.t. y
    curv_loss,_=integrate.dblquad(bs_xx+bs_yy,min_x,max_x,min_x,max_x) #Curvature loss is computed by taking second derivative. 
    return curv_loss
    

In [155]:
#This is just an example.

#Generate random data points
np.random.seed(42)  # For reproducibility
x = np.linspace(0, 10, 100)
y = np.linspace(0, 10, 100)
z = np.random.random((100, 100))

# Create a meshgrid from x and y
X, Y = np.meshgrid(x, y)

# Flatten the meshgrid arrays and z array to pass to bisplrep
x_flat = X.flatten()
y_flat = Y.flatten()
z_flat = z.flatten()

#Define the parameters
k =3 #Degree (both x and y direction)
n = 10 # The number of control points (n+1) 

tx=create_knots(k,n)
ty=create_knots(k,n)

#Create a B-Spline representation using previous determined knots on the data and return the error. 
tck,_=fit_Bspline(x_flat, y_flat, z_flat, k=k, tx=tx, ty=ty)

curv_bs = lambda x,y: bisplev(x,y,tck,2,0)
integrate.dblquad(curv_bs,0,10,0,5)

c:\Anaconda3\envs\IPFD\Lib\site-packages\scipy\integrate\_quadpack_py.py:1272: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


(-1.6160554463795678, 1.0530730754076956e-07)